In [ ]:
#Import packages 
import xarray as xr
import pyrams.xarray
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from glob import glob
from tqdm import tqdm
import pint_xarray

ureg = pint_xarray.unit_registry

import numpy as np
import matplotlib.pyplot as plt
from metpy.calc import mixing_ratio_from_relative_humidity, relative_humidity_from_mixing_ratio
from metpy.units import units
from metpy.constants import g, Rd, Lv, Cp_d, Rv, rho_d
from metpy.calc import temperature_from_potential_temperature, mixing_ratio_from_relative_humidity


In [ ]:
#open the simulation datasets
simlist = [
    '20',
    '200',
    '400',
    '600',
    '800',
    '1000',
]

sims = {}
for sim in simlist:
    data = xr.open_dataset(f'model_data/{sim}.nc')
    data = data.rams.apply_variable_metadata()
    data['time'] = (data.time - data.time[0]) / np.timedelta64(1,'h')
    data.time.attrs['units'] = 'hour'    
    sims[sim] = data
    
for name, ds in tqdm(sims.items()):
    ds = ds.pint.dequantify()

cmap = plt.get_cmap('turbo', len(sims))

#open the sensitivity simulation datasets
simlist2 = [
    '400',
    '400-subs',
    '400-inv',
    '400-cool',
    '400-stable',
    '1000',
    '1000-subs',
    '1000-inv',
    '1000-cool',
    '1000-stable',
]

sims2 = {}
for sim in simlist2:
    data = xr.open_dataset(f'model_data/{sim}.nc')
    data = data.rams.apply_variable_metadata()
    data['time'] = (data.time - data.time[0]) / np.timedelta64(1,'h')
    data.time.attrs['units'] = 'hour'    
    sims2[sim] = data
    
for name, ds in tqdm(sims2.items()):
    ds = ds.pint.dequantify()

cmap2 = plt.get_cmap('tab10', 10)

plots_dir = './plots/'

abc='abcdefghij'

In [ ]:
#Initial* conditions - Need to make this pretty
#Actually, conditions at 0.5 hrs after some model spin up
fig, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=(9,3), constrained_layout=True)
#first two loop over sensitivity tests
for i,(name, ds) in enumerate(sims2.items()):
    if i==1:
        continue
    ds = ds.isel(time=2)
    ds.THETA.plot(y='z', ax=ax1, color=cmap2(i), label='salt'+name)
    if i==4: #because I don't know how to loop over just some of the simulations
        break
ax1.text(0.1, 0.9, 'a)', transform=ax1.transAxes)   
ax1.set_title('')
ax1.legend(["BASE,\nWEAKSUBS","MODINV","COOLER","STABLE"],loc='lower right', fontsize=9)

for i,(name, ds) in enumerate(sims2.items()):
    if i==1:
        continue
    ds = ds.isel(time=2)
    ds.RV.pint.to('g/kg').plot(y='z', ax=ax2, color=cmap2(i), label='salt'+name)
    if i==4:
        break
for i,(name, ds) in enumerate(sims2.items()):
    if i==1:
        continue
    ds = ds.isel(time=2)
    ds.RCP.pint.to('dg/kg').plot(y='z', ax=ax2, color=cmap2(i), label='salt'+name,linestyle='--')
    if i==4:
        break
ax2.set_title('')
ax2.text(0.9, 0.9, 'b)', transform=ax2.transAxes)

#loops over salt profiles
for i,(name, ds) in enumerate(sims.items()):
    ds = ds.isel(time=2)
    ds.SALT_FILM_NP.pint.to('1/mg').plot(y='z', ax=ax3, color=cmap(i), label='salt'+name)
ax3.set_title('')
ax3.legend(ncol=2, fontsize=9)
ax3.text(0.9, 0.9, 'c)', transform=ax3.transAxes)

ax1.set_xlabel('Potential Temperature [K]')
ax1.set_ylabel('Height [m]')
ax2.set_xlabel('Mixing Ratio [g/kg]')
ax2.set_ylabel('')
ax3.set_xlabel('Salt Conc. [mg$^{-1}$]')
ax3.set_ylabel('')

#and finally, save it
plt.savefig(f'{plots_dir}/02.pdf', dpi=500)

In [ ]:
#conda env create -f environment.yml
#conda activate sterzinger-igel-2023-data
#pip install pandas numpy matplotlib tqdm xarray pyrams pint-xarray

In [ ]:
#Define functions to calculate cloud normalized height and BL normalized height and execute them.
def cloudboundary(ds):
    ctoparr = []
    cbotarr = []
    zl, tl = len(ds.z), len(ds.time)
    for t in range(tl):
        dst = ds.isel(time=t)
        top = False
        if dst.RCP.max() >= 1e-5:
            for z in reversed(range(zl)):
                if top and dst.RCP.isel(z=z) < 1e-5 : 
                    cbotarr.append(dst.z[z+1].data)
                    break
                elif not top and dst.RCP.isel(z=z) >= 1e-5:
                    ctoparr.append(dst.z[z].data)
                    top = True
        else:
            cbotarr.append(np.nan)
            ctoparr.append(np.nan)
    # ctoparr
    cloud_top = xr.DataArray(ctoparr, coords=dict(time=ds.time))
    cloud_bot = xr.DataArray(cbotarr, coords=dict(time=ds.time))
    ds['CLOUD_TOP'] = cloud_top
    ds['CLOUD_BOT'] = cloud_bot
    return ds

#Execute the functions
for name,ds in tqdm(sims.items()):
    ds = cloudboundary(ds)
    sims[name] = ds
    
for name, ds in tqdm(sims.items()):
    Z_NORM_CLOUD = np.zeros((len(ds.time), len(ds.z)))
    for t in range(len(ds.time)):
        ds1 = ds.isel(time=t)
        Z_NORM_CLOUD[t,:] = (ds1.z.data - ds1.CLOUD_BOT.data)/(ds1.CLOUD_TOP - ds1.CLOUD_BOT).data

    ds = ds.assign_coords(Z_NORM_CLOUD = (['time', 'z'], Z_NORM_CLOUD))
    
    sims[name] = ds

#Throws a warning about divide by 0, this is okay and expected when cloudtop==cloudbot

In [ ]:
def bltop(ds):
    bltops = []
    # bltopsb = []
    for t in ds.time:
        ds1 = ds.sel(time=t)
        x = ds1.THETA.z
        y = ds1.THETA

        d2y = np.gradient(np.gradient(y))
        zi = np.argmax(d2y)
        bltops.append(ds.z[zi])
        
    darr = xr.DataArray(
        bltops,
        coords = [ds.time.data],
        dims = ["time"]
    )
    ds['BLTOP'] = darr
    return ds

#Execute the functions
for name,ds in tqdm(sims.items()):
    ds = bltop(ds)
    sims[name] = ds
    
for name, ds in tqdm(sims.items()):
    Z_NORM_BLTOP = np.zeros((len(ds.time), len(ds.z)))
    Z_BLTOP = np.zeros((len(ds.time), len(ds.z)))
    for t in range(len(ds.time)):
        ds1 = ds.isel(time=t)
        Z_NORM_BLTOP[t,:] = ((ds1.z.data)/(ds1.BLTOP).data)
        Z_BLTOP[t,:] = ds1.z.data-ds1.BLTOP.data

    ds = ds.assign_coords(Z_NORM_BLTOP = (['time', 'z'], Z_NORM_BLTOP))
    ds = ds.assign_coords(Z_BLTOP = (['time','z'], Z_BLTOP))
    
    sims[name] = ds

In [ ]:
#Execute the functions
for name,ds in tqdm(sims2.items()):
    ds = bltop(ds)
    sims2[name] = ds
    
for name, ds in tqdm(sims2.items()):
    Z_NORM_BLTOP = np.zeros((len(ds.time), len(ds.z)))
    Z_BLTOP = np.zeros((len(ds.time), len(ds.z)))
    for t in range(len(ds.time)):
        ds1 = ds.isel(time=t)
        Z_NORM_BLTOP[t,:] = ((ds1.z.data)/(ds1.BLTOP).data)
        Z_BLTOP[t,:] = ds1.z.data-ds1.BLTOP.data

    ds = ds.assign_coords(Z_NORM_BLTOP = (['time', 'z'], Z_NORM_BLTOP))
    ds = ds.assign_coords(Z_BLTOP = (['time','z'], Z_BLTOP))
    
    sims2[name] = ds

In [ ]:
fig, axs = plt.subplots(3, 2, sharex=True, sharey=True, constrained_layout=True, figsize=(5,5))

for i,((name, ds), ax) in enumerate(zip(sims.items(), axs.flatten())):
    ds = ds.sel(time=slice(2,30))
    RT=ds.RCP+ds.RRP
    p = RT.where(RT >= 0.01 * ureg('g/kg')).pint.to('g/kg').plot(x='time', vmin=0, vmax=.35, ax=ax, add_colorbar=False)     
    p.set_rasterized(True)
    #add single cloud outline contour
    ax.text(0.05,0.85,abc[i]+') salt'+name,transform=ax.transAxes)
    if i%2 != 0: ax.set_ylabel('')
    else: ax.set_ylabel('Height [m]')
    if i not in [4,5]: ax.set_xlabel('')
    else: ax.set_xlabel('Time [hr]')
       
plt.ylim(0,1500)
plt.colorbar(p, ax=axs.ravel().tolist(), shrink=1, label='g kg$^{-1}$')
plt.savefig(f'{plots_dir}/03.pdf', dpi=500)

In [ ]:
fig, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=(10,3), constrained_layout=True)
for i,(name, ds) in enumerate(sims.items()):
    ds = ds.sel(time=slice(2,30))
    (ds.rams.lwp * ureg('m')).pint.to('g/m^2').plot(ax=ax1,x='time', label=name, color=cmap(i))

ax1.set_ylabel('LWP [g m$^{-2}$]')
ax1.set_xlabel('Time [hr]')
ax1.text(0.9, 0.9, 'a)', transform=ax1.transAxes)
    
for i,name in enumerate(sims.keys()):
    ds = sims[name]
    ds = ds.sel(time=slice(2,30))
    ((ds.RRP * ds.DN0).integrate('z') * ureg('m')).pint.to('g/m^2').plot(ax=ax2, label='salt'+name, color=cmap(i))

ax2.text(0.9,0.9,'b)',transform=ax2.transAxes)
ax2.set_xlabel('Time [hr]')
ax2.set_ylabel('RWP [g m$^{-2}$]')

for i,name in enumerate(sims.keys()):
    ds = sims[name].sel(time=slice(2,30))
    bl_mass = ds.SALT_FILM_NP.isel(z=0)*ds.DN0.isel(z=0)
    bl_mass.pint.to('1/cm^3').plot(ax=ax3, color=cmap(i), label='salt'+name)
    (ds.CCP*ds.DN0).where(ds.RCP >= 0.01 *ureg('g/kg')).mean('z').pint.to('1/cm^3').plot(ax=ax3,color=cmap(i), linestyle='--')
 
ax3.text(0.9,0.9,'c)',transform=ax3.transAxes)
ax3.set_ylabel('Number Concentration [# cm$^{-3}$]')
ax3.set_xlabel('Time [hr]')

plt.legend(ncol=1,bbox_to_anchor=(1.05, .8))

plt.savefig(f'{plots_dir}/04.pdf', dpi=500)

In [ ]:
fig, axs = plt.subplots(2, 4, sharey=True, constrained_layout=True, figsize=(8,4))
axs = axs.flatten()

times = [3,12,21,30]

for j in range(4):
#for t,ax in zip(times, axs):
    handles = []
    for i,(name, ds) in enumerate(sims.items()):
        ds1 = ds.sel(time=times[j])
        ax=axs[j]
        p = (ds1.CCP * ds1.DN0).pint.to('1/cm^3').plot(y='Z_BLTOP', ax=axs[j], color=cmap(i), label='salt'+name)
        #ax.set_ylim(-0.1,1.1)
        ax.set_ylim(-300,20)
        ax.set_xlim(0,100)
        handles = handles + p
        ax.set_xlabel('Droplet Conc. [cm$^{-3}$]')
        ax.set_title(f'Time = {times[j]}.0 hr')
    if times[j] in [times[0]]: 
        ax.set_ylabel('Hgt. w.r.t. BL Top [m]')
    else: ax.set_ylabel('')
    
fig.supylabel('(a)\n\n\n\n\n\n\n\n\n\n\n(b)\n',rotation='horizontal')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#for t,ax in zip(times, axs):
for j in range(4):
    for i,(name, ds) in enumerate(sims.items()):
        ds1 = ds.sel(time=times[j])
        ax=axs[j+4]
        v = ds1.rams.cloudradius
        l = v.where(ds1.RCP >= 0.01 * ureg('g/kg')).pint.to('micron').plot(y='Z_BLTOP', ax=axs[j+4], color=cmap(i), label=name)
        #ax.set_ylim(-0.1,1.1)
        ax.set_ylim(-300,20)
        # handles.append(p[0].get_label())
        handles = handles + l
        ax.set_xlabel('Droplet Radius [$\mu$m]')
        ax.set_xlim(0,25)
        ax.set_title('')
    if times[j] in [times[0]]: ax.set_ylabel('Hgt. w.r.t. BL Top [m]')
    else: ax.set_ylabel('')

plt.savefig(f'{plots_dir}/05.pdf', dpi=500)


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(6,6), constrained_layout=True)
axs=axs.flatten()

cp = 1004 * ureg('J/kg/K')

for i, (name,ds) in enumerate(sims.items()):
    ds = ds.sel(time=slice(2,30))
    v = -1* (ds.FTHRD * cp * ds.DN0).where(ds.Z_NORM_CLOUD <= 1,0).where(ds.Z_NORM_CLOUD >= 0,0).integrate('z') * ureg('m')
    v[v==0]=np.nan*ureg('J/s/m^2')
    v.pint.to('W/m^2').plot(ax=axs[0], label=name, color=cmap(i))
axs[0].set_ylabel('Cloud Rad. Flux Diff. [W m$^{-2}$]')
axs[0].set_xlabel('Time [hr]')
axs[0].text(0.9,0.9,'a)',transform=axs[0].transAxes)
#axs[0].set_title('a) Cloud Rad. Flux Difference')

for i,name in enumerate(sims.keys()):
    ds = sims[name]
    ds = ds.sel(time=15, z=slice(0,1200))
    ds.VFLXW.plot(y='z', label=name, color=cmap(i),ax=axs[1])
axs[1].set_xlabel('Vertical Wind Variance [m$^2$ s$^{-2}$]')
axs[1].set_title('')
axs[1].text(0.05,0.9,'b)',transform=axs[1].transAxes)
axs[1].set_ylabel('Height [m]')
axs[1].set_xscale('log')
axs[1].set_xlim((1e-6, 1))

for i,(name, ds) in enumerate(sims.items()):
    ds = ds.sel(time=slice(2,30))
    ds.BLTOP.plot(x='time', label=name, color=cmap(i),ax=axs[2])

axs[2].set_ylabel('Boundary Layer Top Height [m]')
#axs[2].set_title('c) Boundary Layer Top')
axs[2].set_xlabel('Time [hr]')
axs[2].text(0.05,0.9,'c)',transform=axs[2].transAxes)

for i,(name, ds) in enumerate(sims.items()):
    ds = ds.sel(time=15, z=slice(0,1200))
    #v=ds.THETA.gradient('z')*ureg('K/km')
    #v.plot(y='z', label='salt'+name, color=cmap(i),ax=axs[3])
    ds.THETA.plot(y='z', label='salt'+name, color=cmap(i),ax=axs[3])
    
axs[3].set_xlabel('Potential Temperature [K]')
axs[3].set_ylabel('Height [m]')
axs[3].set_title('')
axs[3].text(0.05,0.9,'d)',transform=axs[3].transAxes)
axs[3].legend()

plt.savefig(f'{plots_dir}/06.pdf', dpi=500)

In [ ]:
fig, axs = plt.subplots(1,3, sharey=True, constrained_layout=True, figsize=(7,2.5))

times = [3.0,7.5,12.0,16.5,21.0,25.5,30.0]
colors = plt.get_cmap('viridis', len(times))
sims_include = ['200','600','1000']
j=-1
sims_sub={name:ds for name,ds in sims.items() if name in sims_include}
for ((name,ds), ax) in zip(sims_sub.items(), axs.flatten()):
    j=j+1
    for i,t in enumerate(times):
        ds1 = ds.sel(time=t)
        #aero = (ds1.THETA)
        #aero.plot(y='Z_BLTOP', ax=ax, color=colors(i), label=t)
        aero = (ds1.SALT_FILM_NP)
        aero.pint.to('1/mg').plot(y='Z_BLTOP', ax=ax, color=colors(i), label=t)
        ax.set_ylim(-10, 200)
    
    ax.axhline(1, color='k', linestyle='--')
    ax.set_title(abc[j]+') salt'+name)
    ax.set_ylabel('')   
    ax.set_xlabel('Salt Conc. [# mg$^{-1}$]')

axs[0].set_ylabel('Height w.r.t. BL Top [m]')
axs[0].legend(title='Hour', fontsize=8)
plt.savefig(f'{plots_dir}/07.pdf', dpi=500)

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(8.5,4.5), constrained_layout=True)
for i,(name, ds) in enumerate(sims2.items()):
    ds = ds.sel(time=slice(1,30))
    if i<5:
        (ds.rams.lwp * ureg('m')).pint.to('g/m^2').plot(ax=axs[0,0],x='time', label=name, color=cmap2(i), linewidth=2)
    else:
        (ds.rams.lwp * ureg('m')).pint.to('g/m^2').plot(ax=axs[1,0],x='time', label=name, color=cmap2(i-5), linewidth=2)

axs[0,0].set_ylim(0,80)
axs[1,0].set_ylim(0,150)
axs[0,0].set_ylabel(r'$\bf{salt400}$'' \n ''LWP [g m$^{-2}$]')
axs[1,0].set_ylabel(r'$\bf{salt1000}$'' \n ''LWP [g m$^{-2}$]')
axs[1,0].set_xlabel('Time [hr]')
axs[0,0].set_xlabel('')
axs[0,0].text(0.9, 0.9, 'a)', transform=axs[0,0].transAxes)
axs[1,0].text(0.9, 0.9, 'e)', transform=axs[1,0].transAxes)
    
for i,name in enumerate(sims2.keys()):
    ds = sims2[name]
    ds = ds.sel(time=slice(1,30))
    if i<5:
        ((ds.RRP * ds.DN0).integrate('z') * ureg('m')).pint.to('g/m^2').plot(ax=axs[0,1], label='salt'+name, color=cmap2(i), linewidth=2)
    else: 
        ((ds.RRP * ds.DN0).integrate('z') * ureg('m')).pint.to('g/m^2').plot(ax=axs[1,1], label='salt'+name, color=cmap2(i-5), linewidth=2)
axs[0,1].text(0.9,0.9,'b)',transform=axs[0,1].transAxes)
axs[1,1].text(0.9,0.9,'f)',transform=axs[1,1].transAxes)
axs[1,1].set_xlabel('Time [hr]')
axs[0,1].set_xlabel('')
axs[0,1].set_ylabel('RWP [g m$^{-2}$]')
axs[1,1].set_ylabel('RWP [g m$^{-2}$]')

for i,name in enumerate(sims2.keys()):
    ds = sims2[name].sel(time=slice(1,30))
    if i<5:
        (ds.CCP*ds.DN0).where(ds.RCP >= 0.01 *ureg('g/kg')).mean('z').pint.to('1/cm^3').plot(ax=axs[0,2],color=cmap2(i), linewidth=2, label='salt'+name)
    else:
        (ds.CCP*ds.DN0).where(ds.RCP >= 0.01 *ureg('g/kg')).mean('z').pint.to('1/cm^3').plot(ax=axs[1,2],color=cmap2(i-5), linewidth=2, label='salt'+name)
axs[0,2].text(0.9,0.9,'c)',transform=axs[0,2].transAxes)
axs[1,2].text(0.9,0.9,'g)',transform=axs[1,2].transAxes)
axs[1,2].set_xlabel('Time [hr]')
axs[0,2].set_xlabel('')
axs[1,2].set_ylabel('Droplet Conc. [# cm$^{-3}$]')
axs[0,2].set_ylabel('Droplet Conc. [# cm$^{-3}$]')
axs[0,2].set_ylim(0,45)
axs[1,2].set_ylim(0,120)
#plt.legend(ncol=1,bbox_to_anchor=(1.05, 1))

for i,name in enumerate(sims2.keys()):
    ds = sims2[name].sel(time=30)
    if i<5:
        ds.SALT_FILM_NP.pint.to('1/mg').plot(y='Z_BLTOP', ax=axs[0,3], color=cmap2(i), label='salt'+name)
        ax=axs[0,3]
    else:
        ds.SALT_FILM_NP.pint.to('1/mg').plot(y='Z_BLTOP', ax=axs[1,3], color=cmap2(i-5), label='salt'+name)
        ax=axs[1,3]
     
    ax.set_xlabel('')
    ax.set_ylabel('Height w.r.t. BL Top [m]')
    ax.set_ylim(-10, 200)
    ax.set_title('')

axs[1,3].axhline(1, color='k', linestyle='--') 
axs[0,3].axhline(1, color='k', linestyle='--')
axs[1,3].set_xlabel('Salt Conc. [# mg$^{-3}$]')
axs[0,3].text(0.9,0.9,'d)',transform=axs[0,3].transAxes)
axs[1,3].text(0.9,0.9,'f)',transform=axs[1,3].transAxes)
axs[1,3].legend(['BASE','WEAKSUBS','MODINV','COOLER','STABLE'],fontsize=8)

plt.savefig(f'{plots_dir}/08.pdf', dpi=500)